In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [3]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [8]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')

t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

87.40194582939148


(17435, 37758)

In [6]:
t1=time.time()
cmd="SELECT * FROM %s"%osm_intersections_table_name
cursor.execute(cmd)
intersection=pd.DataFrame(cursor.fetchall())
t2=time.time()
print(t2-t1)

0.43584537506103516


In [25]:
intersection['edge']=intersection.apply(lambda row: (row['osm_begin'],row['osm_end'],row['osm_key']),axis=1)
edges=intersection['edge'].unique()
edges=sorted(edges,key=lambda edge:len(intersection[intersection.edge==edge]))
edges

[(14468319, 1434053206, 0),
 (14468396, 727607080, 0),
 (14468396, 7728949480, 0),
 (14468396, 7816464853, 0),
 (14468399, 14468944, 0),
 (14468466, 61155600, 0),
 (14468467, 5308386915, 0),
 (14470565, 339509156, 0),
 (14470565, 5100643718, 0),
 (14470606, 5877291784, 0),
 (14470606, 5877293692, 0),
 (14471011, 2325146690, 0),
 (14471011, 9831361203, 0),
 (14471044, 9890112443, 0),
 (14471723, 255875884, 0),
 (14472099, 51324493, 0),
 (14472100, 9074944644, 0),
 (14472132, 59733282, 0),
 (15587709, 5582020451, 0),
 (15587788, 15901725, 0),
 (15901725, 256062322, 0),
 (15901775, 3560177481, 0),
 (15901801, 1797878096, 0),
 (15901804, 338458880, 0),
 (15901804, 1885387540, 0),
 (15901832, 3399046081, 0),
 (15904531, 2360732270, 0),
 (15904551, 440728344, 0),
 (15904559, 347509188, 0),
 (15904566, 2312517389, 0),
 (15904622, 3178770718, 0),
 (15904696, 349915162, 0),
 (15904705, 50533209, 0),
 (15904777, 512087990, 0),
 (15904780, 15904782, 0),
 (15904782, 7629680927, 0),
 (15904802, 505

In [ ]:
edge=edges[-12]
edge_df=intersection[intersection.edge==edge]
edge_df=edge_df.sort_values('edge_coordinate')
X=edge_df['edge_coordinate']
Y=edge_df['elevation']
plt.plot(X,Y)

In [72]:
print(np.min(Y),np.max(Y))

20.0 470.0


In [75]:
begin,end=edge[0],edge[1]
print(G_osm.nodes()[begin]['y'],G_osm.nodes()[begin]['x'])
print(G_osm.nodes()[end]['y'],G_osm.nodes()[end]['x'])

42.1372922 8.5996449
42.1442114 8.716983


In [76]:
dY=np.diff(Y)
print(np.sum(np.where(dY>0,dY,0)),np.sum(np.where(dY<0,dY,0)))

420.0 -505.0
